In [1]:
import paramiko
import pandas as pd
from io import StringIO
import json
from shapely.geometry import shape
import pytz
from datetime import datetime, timedelta
from time import sleep
from calendar import monthrange
from multiprocessing import Pool, Process
import requests
from requests.auth import HTTPBasicAuth
import geojson
import geopandas as gpd
import movingpandas as mpd
from datetime import timedelta, datetime
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests

ModuleNotFoundError: No module named 'paramiko'

In [3]:
#init ssh client
client = paramiko.SSHClient()
client.load_system_host_keys()
client.connect('data.opensky-network.org', port=2230, username='username_here', password='password_here

In [4]:
def queryImpala(query):
    #query Impala shell
    stdin, stdout, stderr = client.exec_command(query.replace('\n',''))
    
    error = [line for line in stderr]
    print(error) 
    
    #convert stdout arrays to a dataframe
    data = []
    
    for line in stdout:
        #remove data headers
        if not line.startswith('+'):
            data.append(line.strip('\n').replace(' ', '').strip('|'))
    
    if len(data):
        #remove empty spaces and table formatters
        #temp - StringIO is a dumb way of converting the text into dataframe
        df = pd.read_csv(StringIO('\n'.join(data)), sep = '|')
        return df
    else:
        return pd.DataFrame()

In [5]:
#generate bounds from geojson file
with open('./manhattan_boundary.geojson') as geojson:
    feature = json.load(geojson)['features'][0]['geometry']
    mn_bounds = shape(feature)

bounds = mn_bounds.bounds

def getMonth(month):
    #init ssh client
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.connect('data.opensky-network.org', port=2230, username='USERNAME', password='PASSWORD    #generate queries for every data in month, query, then add to dataframe.
    global month_rows
    month_rows = pd.DataFrame()
    timezone = pytz.timezone("America/New_York")

    for day in range(1,2):
        for hour in range(0, 24):
            utc = datetime(2022, month, day, hour, 0, 0).astimezone(timezone)

            start = int(utc.timestamp())
            end = int((utc + timedelta(hours = 1)).timestamp())

            start_hour = start -(start%3600) 
            #build sql query where time is that hour, in bounds, 
            #below 762 meters (2,500 feet), and not on the ground
            query = f"""--query=SELECT * FROM state_vectors_data4 
                WHERE hour={start_hour}
                AND time % 4 = 0
                AND lon >= {bounds[0]} AND lat >= {bounds[1]} AND lon <= {bounds[2]} AND lat <= {bounds[3]} 
                AND geoaltitude <= 762
                AND onground is false
            """

            df = queryImpala(query)

            print(f'{utc.strftime("%b %d %Y %H:%M")} with {len(df)} rows')

            month_rows = pd.concat([month_rows, df])

            sleep(5)
               
    month_rows.to_csv(f'./{month}_2022_airport_test.csv', index = False)

In [7]:
getMonth(7)

['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 00:00 with 78 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 01:00 with 303 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 02:00 with 105 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 03:00 with 0 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 04:00 with 0 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 05:00 with 65 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 06:00 with 3074 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 07:00 with 2113 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 08:00 with 1273 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 09:00 with 3314 rows
['Starting Impala Shell without Kerberos authentication\n']
Jul 01 2022 10:00 with

In [ ]:
#client = paramiko.SSHClient()
#client.load_system_host_keys()
#client.connect('data.opensky-network.org', port=2230, username='zhi_betanyc', password=';A8=[C.e9Ybt')
flight_info_list = []
for icao in month_rows[2]:
    query2 = f"""-- query=SELECT icao24, estdepartureairport,  estarrivalairport from flights_data4 WHERE icao24 = {icao}
    """

In [8]:

res = month_rows.icao24.unique()
print(res)

['aadffd' 'a6178b' 'a69e14' 'acb1f5' 'a02b97' 'ac6c45' 'a22ee9' 'icao24'
 'ad340b' 'ac1dbc' 'a51ebb' 'a81624' 'aa8e7f' 'a928da' 'ac234a' 'a3b8e8'
 'a8da77' 'a0e5ee' 'acbcfd' 'a9923b' 'a4d45c' 'a4b205' 'a92e73' 'ac3d2a'
 'a5fc87' 'a4bd2a' 'a98b8b' 'aa8548' 'ab5017' 'a81f62' 'a43fba' 'ab6800'
 'a8b802' 'a50106' 'aab316' 'a2cbbd' 'a79f25' 'ad35e0' 'a466a1' 'a9ba85'
 'a1d45a' 'c00b7c' 'ab282b' 'a1d49e' 'ac2bd5' 'a4c4e1' 'acda2c' 'c069b9'
 'ab35a9' 'a34825' 'a62bdc' 'a2fe3c' 'abdc96' 'a06e71' 'a07f60' 'a6152a'
 'a422f7' 'a486b3' 'a08714' 'aaf8f1' 'a9a037' 'ab6394' 'a52cc8' 'a56e0d'
 'aca79d' 'a31bd5' 'a2c0b7' 'a1d0e7' 'a13f8b' 'a91ab7' 'aa9723' 'a2adad'
 'a35de5' 'a12d4b' 'a56e48' 'a2270f' 'a1ed17' 'a1ad1f' 'a2d518' 'aadc2b'
 'a01bbc' 'a066a3' 'a19dff' 'adf668' 'ada521' 'a47aac' 'a4c03c' 'a14c9b'
 'a46bd0' 'ae10e8' 'a81b2a' 'a8423c' 'a24720' 'a1f0b4' 'a45f37' 'a3ad5a'
 'accb65' 'a466c7' 'a23fb2' 'aa02ab' 'a0aab8' 'a2f710' 'a8d3af' 'a5e880'
 'ac2afe' 'a02648' 'ae6ab8' 'a83876' 'a1e946' 'ab05

In [ ]:
client.close()